### Logist Regression

#### Deduction
Input: $$
X =
  \begin{bmatrix}
  1 & 2 & \cdots & 3\\
  4 & 4 & \cdots & 6\\
  7 & 8 & \cdots & 9\\
  \vdots & \vdots & \ddots & \vdots \\
  11 & 12& \cdots & 13\\
  \end{bmatrix}_{m*n}
,
Y =
  \begin{bmatrix}
  1\\
  0\\
  1\\
  \vdots\\
  1\\
  \end{bmatrix}_{m*1}
$$

Hypothesis: $$
  H_\theta(x) = g(z) = \frac{1}{1+e^{-z}} = \frac{e^z}{1+e^z}
  = \frac{1}{1+e^{-\theta^T X}} \tag{1}
$$

  
Linear function: $$
  z = \theta^T X, \quad z \in (-\infty, \infty)
$$

Logist/Sigmoid function: $$
  g(z) = \frac{1}{1+e^{-z}} = \frac{e^z}{1+e^z}, \quad g(z) \in (0, 1)
$$

| z $\geq$ 0 | z < 0 |
| :---: | :---: |
| $\theta^T X$ $\geq$ 0 | $\theta^T X$ < 0 |
| $h_\theta(x)$ $\geq$ 0.5 | $h_\theta(x)$ < 0.5 |
| **g(z)** $\geq$ 0.5 | **g(z)** < 0.5 |
| **y** = 1 | **y** = 0 |

![Sigmoid](./img/sigmoid.jpg)

```python
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

z = np.linspace(-10, 10, 1000)
g = 1 / (1 + np.e ** -z)

fig, ax = plt.subplots(figsize=(5, 3))
plt.xlabel('z')
plt.ylabel('g(z)')
sns.lineplot(x='z', y='g(z)', ax=ax, data={'z': z, 'g(z)': g})
```


$H_\theta(x)$ 被Sigmoid函数归一化到(0,1)，表示结果分类结果为True的概率。

$$
\begin{align}
P(y=1|x;\theta) &= H_\theta(x) \\
P(y=0|x;\theta) &= 1 - H_\theta(x)
\end{align}
$$

Assume $y|x;\theta \sim Bernoulli(1, p)$，合并出y的概率质量函数（y在各特定取值上的概率总和为1）:<br> $$
\begin{align}
  P(y|x;\theta) &= p^y (1-p)^{1-y} \\
                &= \begin{cases} p, \quad &y = 1 \\
                                 1 - p, \quad &y = 0
                   \end{cases} \\
                &= (H_\theta(x))^y (1 - H_\theta(x))^{1-y} \tag{2}
\end{align}
$$


Maximum likelihood function是数据集出现概率最大的函数，一个事件已经观察到的维度发生的概率越大，那么未观察到的维度（误差）发生的概率越小。

所有样本的贡献相互独立, 数据集的概率是每一个样本概率之乘积: $$
  L(\theta) = \prod_{j=0}^{m} p(y^j|x^j;\theta)
$$

获得最大化似然的参数, 是对$\theta$的参数估计: $$
\begin{align}
  \theta^* = \operatorname*{arg\ max}_{\theta}\ L(\theta) &= \prod_{j=0}^{m} p(y^j|x^j;\theta) \\
  &= \prod_{j=0}^{m} (H_\theta(x)^j)^y \, (1-H_\theta(x)^j)^{1-y} \tag{3}
\end{align}
$$


Log transformation，1.连乘变成连加方便求导，2.概率很小时候连乘容易等于0: $$
\begin{align}
  \ell(\theta) &= \log(L(\theta)) \\
  &= \sum_{j=0}^{m} y^j\ log(H_\theta(x)^j) + (1-y^j)\ log(1-H_\theta(x)^j) \tag{4} \\
\end{align}
$$

Binary cross engropy(minimize, 预测分布越接近真实分布): $$
  -\ell(\theta) = -\sum_{j=0}^{m} y^j\ log(H_\theta(x)^j) + (1-y^j)\ log(1-H_\theta(x)^j)
$$

Partial Derivative for every $\theta$: $$
\begin{align}
  \frac{\partial \ell(\theta)}{\partial \theta_i} &= \sum_{j=0}^{m} (y^j \frac{1}{H_\theta(x)^j} + (1-y^j) \frac{0-1}{1-H_\theta(x)^j}) \frac{\partial}{\partial \theta_i} H_\theta(x)^j \\
        &= \sum_{j=0}^{m} (y^j \frac{1}{H_\theta(x)^j} - (1-y^j) \frac{1}{1-H_\theta(x)^j}) \frac{\partial}{\partial \theta_i} H_\theta(x)^j \\
  \because g'(z) &= \frac{e^{-z}}{(1+e^{-z})^2} \\
        &= \frac{1}{1+e^{-z}} \cdot \frac{e^{-z}}{1+e^{-z}} \\
        &= g(z) \cdot (1-g(z)) \\
  \therefore \frac{\partial \ell(\theta)}{\partial \theta_i} &= \sum_{j=0}^{m} (y^j \frac{1}{H_\theta(x)^j} - (1-y^j) \frac{1}{1-H_\theta(x)^j})\ H_\theta(x)^j\ (1-H_\theta(x)^j)\ x_i^j \\
  &= \sum_{j=0}^{m} (y^j(1-H_\theta(x)^j) - (1-y^j)H_\theta(x)^j)\ x_i^j \\
  &= \sum_{j=0}^{m} (y^j - H_\theta(x)^j)\ x_i^j \tag{5}
\end{align}
$$

Gradient ascent to optimise $\theta$: $$
\begin{align}
  \theta_i &= \theta_i + \frac{\partial log(L(\theta))}{\partial \theta_i} \\
           &= \theta_i + \frac{\partial \ell(\theta)}{\partial \theta_i} \\
           &= \theta_i + \sum\limits_{j=0}^m (y^j - H_\theta(x)^j) x_i^j
\end{align}
$$

Gradient decend: $$
\theta_i = \theta_i - \sum\limits_{j=0}^m (H_\theta(x)^j - y^j) x_i^j \tag{6}
$$

#### Analysis

1. 设定阈值。Logist regression 计算出的是一个类似概率的值，不一定大于0.5就判断为正，可以根据实际情况设定。
2. 置信度，除了预测出分类，还能给出分类结果的置信度。
3. 对数据倾斜敏感，正负数据比相差悬殊时预测效果不好。损失函数二值交叉熵越小，代表预测分布越接近真实分布，预测越准确。最大似然估计，是所有样本被预测正确的最大概率。样本集正例只有几个，几乎都是负例，则很容易预测准确率很高，但正例预测对的概率很低，此时要对数据重采样/欠采样。

#### Logist Function
因变量和自变量之间的多元线性关系：$Y = \theta_0 x_0 + \theta_1 x_1 + ... + \theta_n x_n, \quad \sum\limits_{i=0}^n \theta_i x_i \in (-\infty, +\infty) $

想利用线性回归模型演化出逻辑回归，要把$(-\infty, +\infty)$的范围归一化到(0,1)上，对数函数是把(0,1)的自变量映射到$(-\infty, 0)$上，$log(Y) = \theta_0 x_0 + \theta_1 x_1 + ... + \theta_n x_n$，这时如果$Y \in (0,1)，log(Y) \in (-\infty, 0)$，还少一半对称的范围。

![Log](./img/log.png)
```python
import numpy as np
from bokeh.plotting import figure, show
x = np.linspace(-10, 10, 1000)
y = np.log(x)
p = figure(title="log(x)", x_axis_label='x', x_range=(-10,10), y_range=(-2,2), plot_height=300)
p.line(x, y, line_width=2)
show(p)
```

几率（odd）是事件发生与不发生概率的比，假设事件E发生的概率为p：$$
odd(E) = \frac{p}{1-p} = \frac{1}{\frac{1}{p}-1} \\
  \because p \in (0,1) \\
  \therefore \frac{p}{1-p} \in (0, +\infty) => log(\frac{p}{1-p}) \in (-\infty, +\infty)
$$

![Odd](./img/odd.png)
![Logit](./img/logit.png)

得出logit变换：$$log(\frac{p}{1-p}) = \theta_0 x_0 + \theta_1 x_1 + ... + \theta_n x_n = \sum\limits_{i=0}^n \theta_i x_i \tag{7}
$$
log是指数变换，it是odd函数，logit变换就是对odd函数做指数变换。

对logit变换函数求反函数，得出logist function：$$
\frac{p}{1-p} = e^{\theta^T X} \\
p = \frac{e^{\theta^T X}}{1+e^{\theta^T X}}
$$

#### Maximum likelihood function $^{[1]}$
概率是某事件发生之前，根据环境参数对其发生可能性的预测。比如抛硬币之前，根据硬币是密度均匀分布推测正反面概率都是50%。

似然是某事件发生后，根据确定的结果推测产生结果的环境参数。比如抛1000次硬币，496次朝上，504次朝下，可以大致推断出硬币密度分布均匀，抛硬币的人没有作弊。

如果用$\theta$表示环境参数，x表示结果，概率可表示为，$P(x|\theta)$，似然可以表示为，$L(\theta|x)$.

最大似然估计函数是样本的函数，根据样本的观测值$x_1,x_2,...x_n,$带入统计表达式，得到最大似然估计值。


### References
[1] [最大似然估计、n阶矩、协方差（矩阵）、（多元）高斯分布 学习摘要](https://www.cnblogs.com/boceng/p/11358480.html)